In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import datacorral as dc

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
dc_client = dc.Client()

In [ ]:
def make_plot(df_dataset):
    # compute sizes
    # proportion of bounding boxes whose size <= 10 <- which also implies some pixels will have their location offset > 10
    df_target_bb_size = []
    for _, row in df_dataset.iterrows():
        bounding_boxes = row['bounding_boxes']
        sizes = {'stem': [], 'internal_loop': [], 'hairpin_loop': []}
        for _, (siz_x, siz_y), struct_type in bounding_boxes:
            if struct_type in sizes:
                sizes[struct_type].append(max(siz_x, siz_y))
        # summary
        df_target_bb_size.append({k: np.sum(np.asarray(v) <= 10)/float(len(v)) for k, v in sizes.items()})

    df_target_bb_size = pd.DataFrame(df_target_bb_size)
    # make plot
    fig = make_subplots(rows=1, cols=1)
    bb2color = {
        'stem': px.colors.qualitative.Plotly[0],
        'internal_loop': px.colors.qualitative.Plotly[1],
        'hairpin_loop': px.colors.qualitative.Plotly[2],
    }

    for bb_type in ['stem', 'internal_loop', 'hairpin_loop']:
        fig.append_trace(go.Histogram(x=df_target_bb_size[bb_type],
                                      name=bb_type, nbinsx=20, marker_color=bb2color[bb_type],
                                      histnorm='percent'), 1, 1)

    fig.update_layout(
        autosize=False,
        width=600,
        height=400,
        title='target: proportion of boxes with size <= 10',
    )        
    return fig

In [ ]:
df_bprna = pd.read_pickle(dc_client.get_path('DmNgdP'), compression='gzip')
df_rand = pd.read_pickle(dc_client.get_path('ZQi8RT'), compression='gzip')

In [ ]:
fig_bprna = make_plot(df_bprna)
fig_bprna.update_layout(
    title="bprna {}".format(fig_bprna.layout['title']['text']),
)   

In [ ]:
fig_rand = make_plot(df_rand)
fig_rand.update_layout(
    title="rand seq {}".format(fig_bprna.layout['title']['text']),
)   